## 混部集群性能分析
    混部：在线电商+离线ODPS作业
    
    在线电商：容器化/JAVA应用等，数量好几万
    离线ODPS作业：每天几十万-几百万作业
    
    
    本脚本为一个集群（AY49C）的混部性能数据

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import time
%matplotlib inline
import matplotlib.pyplot as plt

sys.path.append("../../../")

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('max_colwidth',4000)
pd.options.display.expand_frame_repr = False

%load_ext autoreload
%autoreload 2



In [ ]:
from pets.core.data_loader.res_eff.data_source import get_vm_pdf, get_nc_pdf
from pets.core.data_loader.walle.data_source import get_walle_odps_host_cpu_util, get_walle_online_offline_cpu_util, get_walle_odps_container_cpu_util
import pets.core.data_loader.sunfire.data_source as sunfire
import pets.core.data_loader.kmon.data_source as kmon
import pets.core.data_loader.fuxi.data_source as fuxi

In [ ]:
def trans(data):
#     print(np.std(data))
    return (data - np.mean(data)) / np.std(data)

In [ ]:
machine = pd.read_csv("/data/zkp/m2.csv")

In [ ]:
instance = pd.read_csv("/data/zkp/m2_instance.csv")
instance.fillna(100, inplace=True)

In [ ]:
instance.shape

In [ ]:
ins_start = instance.start_time.values
ins_end = instance.end_time.values
ins_usage = instance.fuxi_avg_cpu.values * instance.total_time_seconds.values
ins_avg_usage = instance.fuxi_avg_cpu.values

In [ ]:
ins_data = [0 for i in range(np.max(ins_end) - np.min(ins_start))]
ins_data_degra = [0 for i in range(np.max(ins_end) - np.min(ins_start))]        
ins_data_hour = [0 for i in range(4319)]

for i in range(ins_start.shape[0]):
    for k in range(ins_end[i] - ins_start[i]):
        ins_data_degra[ins_start[i] - ins_start[0] + k] += ins_avg_usage[i]
        ins_data[ins_start[i] - ins_start[0] + k] += (ins_usage[i])
        
        
# for i in range(ins_start.shape[0]):
#     for k in range(int((ins_end[i] - ins_start[i]) / 60)+1):
#         index = int((ins_start[i] - 1645027200) / 60) + k
#         if index >= 0 and index < 4319:
#             ins_data_hour[index] += (ins_avg_usage[i])

In [ ]:
plt.plot(range(4319), ins_data_hour)

In [ ]:
service = pd.read_csv("/data/zkp/m2_service.csv")
ip = service.container_ip.unique()
for i in ip:
    temp = service[service.container_ip == i]
    if not np.isnan(temp.rt__ms_Q.values[0]) and not np.isnan(temp.rt__ms_Q.values[2]):
        print("\"" + str(i) + "\"", end=', ')

In [ ]:
LSR_ip = ["33.42.152.29", "33.43.231.95", "33.61.106.132"]

In [ ]:
service = pd.read_csv("/data/zkp/m2_service.csv")
service.fillna(0, inplace=True)
target_ip = ["33.4.61.113", "33.43.48.112", "33.43.51.85", "33.43.77.121"]

In [ ]:
for ip in target_ip:
    temp = service[service.container_ip==ip]
    http = temp[temp.throughput_type=="HTTP"].qps__Q_s.values
    hsf = temp[temp.throughput_type=="HSF"].qps__Q_s.values
#     print(np.bincount(np.isnan(hsf)))
    hsf_c = temp[temp.throughput_type=="HSF_Consumer"].qps__Q_s.values
    cpu = temp[temp.throughput_type=="HTTP"].cpu_util__pct.values
    print(ip)
    plt.figure(figsize=(20, 8))
#     plt.plot(range(http.shape[0]), cpu)
    plt.plot(range(http.shape[0]), http)
#     plt.plot(range(hsf.shape[0]), hsf)
#     plt.plot(range(hsf_c.shape[0]), hsf_c)
    
#     plt.yscale("log")
#     plt.plot(range(hsf.shape[0]), trans(temp[temp.throughput_type=="HTTP"].cpu_util__pct))
    plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
for ip in [target_ip[1], target_ip[2], target_ip[13]]:
    temp = service[service.container_ip==ip]
    temp = temp[temp.throughput_type=="HTTP"]
    y = temp.rt__ms_Q / temp.qps__Q_s
    plt.plot(range(temp.shape[0]), (y - np.mean(y)) / np.std(y))

In [ ]:
# plt.figure(figsize=(20, 15))
# plt.plot(range(np.min(ins_start), np.max(ins_end), 10), trans(ins_data[::10]))
# plt.plot(range(np.min(ins_start), np.max(ins_end), 500), trans(ins_data_degra[::500]))


test_service = service[service.container_ip==LSR_ip[2]]
test_http = test_service[test_service.throughput_type=="HTTP"]
test_hsf = test_service[test_service.throughput_type=="HSF"]
test_hsf_c = test_service[test_service.throughput_type=="HSF_Consumer"]
test_timestamp = test_http.sample_time__m.apply(lambda x:time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))).values
print(test_timestamp[0], test_timestamp[-1])
# test = test_http.qps__Q_s.values * test_http.rt__ms_Q.values + test_hsf.qps__Q_s.values * test_hsf.rt__ms_Q.values + test_hsf_c.qps__Q_s.values * test_hsf_c.rt__ms_Q.values
# plt.plot(test_timestamp[::10], trans(test[::10]))
# plt.plot(test_timestamp[::10], trans((test_hsf.rt__ms_Q)[::10]))
# plt.plot(test_timestamp[::10], trans(test_http.cpu_util__pct[::10]))
# test_rt = test_hsf.rt__ms_Q.values
# test_qps = np.exp(test_hsf.qps__Q_s.values)
# plt.plot(test_timestamp, trans(test_rt / test_qps))
# plt.plot(test_timestamp[::10], trans(test_http.qps__Q_s.values[::10] / test_http.cpu_util__pct.values[::10]))
# plt.plot(test_timestamp[::10], trans((test_http.qps__Q_s.values * test_http.rt__ms_Q.values)[::10] / test_hsf.cpu_util__pct[::10]))
# plt.plot(test_timestamp[::1], trans(test_http.rt__ms_Q.values[::1]))
# print(np.corrcoef(test, machine.cpu_util__pct.values))
# plt.plot(machine.sample_time.values[::10], trans(machine.cpu_util__pct.values[::10]))
print(machine.sample_time.values[0], machine.sample_time.values[-1])
# plt.xlim(1.644e9+150000, 1.644e9+250000)
# plt.ylim(0, 5)


In [ ]:
nc_cpu = machine.cpu_util__pct.values
nc_time = machine.sample_time.values
new_nc_cpu = []
cur_time = 1645027200
for i in range(len(nc_time)):
    if nc_time[i] > cur_time:
        new_nc_cpu.append(nc_cpu[i])
        cur_time += 60
    if nc_time[i] == cur_time:
        new_nc_cpu.append(nc_cpu[i])
        cur_time += 60
plt.plot(range(len(new_nc_cpu)), new_nc_cpu)
new_nc_cpu = np.asarray(new_nc_cpu[:-1])

In [ ]:
qps1 = test_http.qps__Q_s.values
qps2 = test_hsf.qps__Q_s.values
qps3 = test_hsf_c.qps__Q_s.values
ins = np.asarray(ins_data_hour)
# rt1 = test_http.rt__ms_Q.values
# rt2 = test_hsf.rt__ms_Q.values
# rt3 = test_hsf_c.rt__ms_Q.values
X_data = np.asarray([qps1, qps2, qps3, new_nc_cpu]).T
print(X_data.shape)
y_data = test_http.cpu_util__pct.values
model = LinearRegression()
model.fit(X_data, y_data)
print(model.score(X_data, y_data))
print(model.coef_, model.intercept_)

In [ ]:
predict = model.predict(X_data)
I_B = []
for i in range(ins.shape[0]):
    I_B.append((y_data[i] - predict[i]) / ins[i] if ins[i] != 0 else 0)
I_B = np.asarray(I_B)
I_B

In [ ]:
# I_B = (I_B - np.min(I_B)) / (np.max(I_B) - np.min(I_B))

In [ ]:
rt1 = test_http.rt__ms_Q.values
rt2 = test_hsf.rt__ms_Q.values
rt3 = test_hsf_c.rt__ms_Q.values
print(rt1.shape, rt1, rt2, rt3)

In [ ]:
def CDFy(data):
    y = 100. * np.arange(len(data))/(len(data)-1)
    return y

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(range(I_B.shape[0]), I_B)
print(np.percentile(I_B, 95), np.percentile(I_B, 5))

In [ ]:
low = []
high = []
for i in range(I_B.shape[0]):
    if I_B[i] > np.percentile(I_B, 80):
        high.append(rt2[i])
    elif I_B[i] <= np.percentile(I_B, 20):
        low.append(rt2[i])
high = sorted(high)
low = sorted(low)
plt.figure(figsize=(10, 8))
plt.plot(low, CDFy(low))
plt.plot(high, CDFy(high))

In [ ]:
mp = {}
for i in range(test_timestamp.shape[0]):
    if test_timestamp[i] not in mp:
        mp[test_timestamp[i]] = []
    mp[test_timestamp[i]].append(test[i])
    mp[test_timestamp[i]].append(test_http.cpu_util__pct.values[i])
for i in range(machine.shape[0]):
    if machine.sample_time.values[i] in mp:
        mp[machine.sample_time.values[i]].append(machine.cpu_util__pct.values[i])

In [ ]:
data = []
data_time = []
for key in mp:
    if len(mp[key]) == 3:
        data.append(mp[key])
        data_time.append(key)
data = np.asarray(data)
data_time = np.asarray(data_time)

LOAD, CU, MU

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
model = LinearRegression()
X_data = data[:, 1:3]
y_data = data[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
plt.figure(figsize=(20, 8))
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))
plt.scatter(y_test, y_predict)
print(model.coef_, model.intercept_)
# plt.xlabel("{:.1f} * MU {:.1f} * CU {:.1f}".format(model.coef_[0], model.coef_[1], model.intercept_), fontsize=15)
# plt.ylabel("LOAD", fontsize=15)

In [ ]:
data_pre = model.predict(X_data)
error = []
for i in range(data_pre.shape[0]):
    if y_data[i]:
        error.append(abs(y_data[i] - data_pre[i]) / y_data[i])
    else:
        error.append(1)
# error = sorted(error)
# for i in range(len(error)):
#     if error[i] > 0.2:
#         print(i/len(error))
#         break


In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(range(np.min(ins_start), np.max(ins_end)), np.asarray(ins_data)/1e5)
# plt.plot(test_timestamp[::10], trans((test_http.cpu_util__pct)[::10]))
plt.plot(machine.sample_time.values[::20], machine.cpu_util__pct.values[::20])
# plt.plot(test_timestamp[::10], trans(test_hsf.qps__Q_s.values[::10]))
plt.vlines(1645027200+86400*2, ymin=-10, ymax=90, colors="red")
plt.vlines(1645027200+86400, ymin=-10, ymax=90, colors="red")
# plt.plot(data_time, error)
plt.ylim(-10, 90)
plt.xlabel("time", fontsize=15)
plt.ylabel("CPU", fontsize=15)
plt.legend(["batch usage(1e5)", "machine CPU"], fontsize=15)

In [ ]:
### 离线作业性能数据
    ay49c_tubo_server 为离线分组

In [ ]:
plt.plot(range(np.min(ins_start), np.max(ins_end)), ins_data)
# plt.plot(range(np.min(ins_start), np.max(ins_end)), ins_data_degra)
print(np.corrcoef(ins_data, ins_data_degra))

In [ ]:
lsmachine = pd.read_csv("./lsmachine.csv")
lsmachine.fillna(0, inplace=True)
lsservice = pd.read_csv("./lsservice.csv")
lsservice.fillna(0, inplace=True)
target_machine = ["i-8vbb15063hgwnnj72thh", "i-8vb9ul5xec4svv1bv1h0", "i-8vb4p5hyvakr5me0hij1", "i-8vb9qrifimm00x67wj12", "i-8vbcaknvxr6wbba8tlsu", "i-8vb2t1xpt6kzgz2brw9h", "i-8vb13matvy02bw36lch3", "i-8vb9bx27cup4tevcgur7", "i-8vbf565jz2vjsriw73ke", "i-8vb19vzmoywz7xqtuvdz"]
ma_meta = pd.read_csv("./pod_meta.csv")

In [ ]:
# ma_meta.groupby(by=ma_meta.nc_sn).agg("count").sort_values(by=["container_ip"])
ma_meta[ma_meta.nc_sn.isin(target_machine)].groupby(by=ma_meta.nc_sn).cpu_set_mode.value_counts()

In [ ]:
for sn in target_machine:
    machine = lsmachine[lsmachine.nc_sn==sn]
    nc_cpu = machine.cpu_util__pct.values
    nc_time = machine.sample_time.values
    new_nc_cpu = []
    cur_time = 1644120000
    for i in range(len(nc_time)):
        if nc_time[i] > cur_time:
            new_nc_cpu.append(nc_cpu[i])
            cur_time += 60
        if nc_time[i] == cur_time:
            new_nc_cpu.append(nc_cpu[i])
            cur_time += 60
    if len(new_nc_cpu) != 61: 
        continue
#     plt.plot(range(0, len(new_nc_cpu)*2, 2), new_nc_cpu)
#     plt.plot(range(nc_cpu.shape[0]), nc_cpu)
#     plt.show()
    new_nc_cpu = np.asarray(new_nc_cpu)
    temp = ma_meta[ma_meta.nc_sn==sn]
    target_ip = temp[temp.qos_class=="LS"].container_ip.unique().tolist()
#     print(target_ip)
    for ip in target_ip:
        test_service = lsservice[lsservice.container_ip==ip]
        test_http = test_service[test_service.throughput_type=="HTTP"]
        test_hsf = test_service[test_service.throughput_type=="HSF"]
        test_hsf_c = test_service[test_service.throughput_type=="HSF_Consumer"]
        test_timestamp = test_http.sample_time__m.apply(lambda x:time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))).values
        qps1 = test_http.qps__Q_s.values
        qps2 = test_hsf.qps__Q_s.values
        qps3 = test_hsf_c.qps__Q_s.values
        rt1 = test_http.rt__ms_Q.values
        rt2 = test_hsf.rt__ms_Q.values
        rt3 = test_hsf_c.rt__ms_Q.values
        test_cpu = test_http.cpu_util__pct.values
        if ip == "33.3.227.61":
            plt.plot(nc_time, trans(nc_cpu))
            print(test_cpu)
            plt.plot(test_timestamp, trans(test_cpu))
            plt.show()
#         plt.scatter(rt2, np.exp(test_cpu))
#         plt.show()
#         print(np.corrcoef(new_nc_cpu, qps2 / test_http.cpu_util__pct))
#         plt.show()
#         X_data = np.asarray([qps1*rt1, qps2*rt2, qps3*rt3]).T
#         if X_data.shape[0] != 61:
#             continue
#         y_data = test_cpu
#         model = LinearRegression()
#         model.fit(X_data, y_data)
#         print(model.score(X_data, y_data), end=',')
#         print(model.coef_, end=',')
#         print(np.mean(y_data))
